In [21]:
import torch
import torch.nn as nn
import pandas as pd
import matplotlib.pyplot as plt
from torch import optim

In [6]:
from argparse import Namespace

modelArgs = Namespace(
    
data_csv = 'timeSeriesData.csv',
model_state_file = "model.pth",
seed = 1300,
learning_rate = 0.001,
# dropout_p = 0.1,
batch_size = 128,
num_epochs = 40,
cuda = False,

);
device = "cuda" if modelArgs.cuda else "cpu"

In [8]:
df = pd.read_csv("data/DailyDelhiClimateTrain.csv");
df.tail()

,date,meantemp,humidity,wind_speed,meanpressure
1457,2016-12-28,17.217391,68.043478,3.547826,1015.565217
1458,2016-12-29,15.238095,87.857143,6.000000,1016.904762
1459,2016-12-30,14.095238,89.666667,6.266667,1017.904762
1460,2016-12-31,15.052632,87.000000,7.325000,1016.100000
1461,2017-01-01,10.000000,100.000000,0.000000,1016.000000


##### Creating the data loader #####

In [19]:
from torch.utils.data import Dataset, DataLoader

class DataSet(Dataset):
    def __init__(self , dataFrame:pd.DataFrame , x_columns:list , y_column:list) -> None:
        super().__init__();
        self.x_data = dataFrame[x_columns];
        self.y_data = dataFrame[y_column];
    
    
    def __getitem__(self, index):
        x_data = torch.tensor(self.x_data.iloc[index].to_numpy());
        y_data = torch.tensor(self.y_data.iloc[index].to_numpy());
        return x_data , y_data;

    def __len__(self):
        return len(self.x_data);

Creating DataLoader

In [20]:
dataLoader = DataLoader(dataset=DataSet(dataFrame=df , x_columns= ['humidity','wind_speed','meanpressure'] ,y_column=['meantemp'] ))
for item in dataLoader:
    print(item)

[tensor([[  84.5000,    0.0000, 1015.6667]], dtype=torch.float64), tensor([[10.]], dtype=torch.float64)]
[tensor([[  92.0000,    2.9800, 1017.8000]], dtype=torch.float64), tensor([[7.4000]], dtype=torch.float64)]
[tensor([[  87.0000,    4.6333, 1018.6667]], dtype=torch.float64), tensor([[7.1667]], dtype=torch.float64)]
[tensor([[  71.3333,    1.2333, 1017.1667]], dtype=torch.float64), tensor([[8.6667]], dtype=torch.float64)]
[tensor([[  86.8333,    3.7000, 1016.5000]], dtype=torch.float64), tensor([[6.]], dtype=torch.float64)]
[tensor([[  82.8000,    1.4800, 1018.0000]], dtype=torch.float64), tensor([[7.]], dtype=torch.float64)]
[tensor([[  78.6000,    6.3000, 1020.0000]], dtype=torch.float64), tensor([[7.]], dtype=torch.float64)]
[tensor([[  63.7143,    7.1429, 1018.7143]], dtype=torch.float64), tensor([[8.8571]], dtype=torch.float64)]
[tensor([[  51.2500,   12.5000, 1017.0000]], dtype=torch.float64), tensor([[14.]], dtype=torch.float64)]
[tensor([[  62.0000,    7.4000, 1015.6667]], d

In [24]:
class Rnn(nn.Module):

    def __init__(self,input_size , hidden_size , output_size , batch_size = 40):

        super(Rnn,self).__init__();
        self.hidden_size = hidden_size;
        self.input_size = input_size;
        self.output_size = output_size;
        self.batch_size = batch_size
        self.i2h = nn.Linear(self.input_size + self.hidden_size , self.hidden_size);
        self.i2o = nn.Linear(self.input_size + self.hidden_size , self.output_size);
        self.sigmoid = nn.Sigmoid();
    
    def forward(self,input_tensor,hidden_tensor):
        print(f'input_tensor {input_tensor} with size == {input_tensor.size()} \n');
        print(f'hidden_tensor {hidden_tensor} with size == {hidden_tensor.size()} \n');

        combined = torch.cat((input_tensor , hidden_tensor) , 1);
        print(f'combined_tensor {combined} with size == {combined.size()} \n');
        hidden = self.i2h(combined)
        output = self.sigmoid(self.i2o(combined));
        return output,hidden;


    def init_hidden(self, batch_size = 1):
        return torch.zeros(batch_size,self.hidden_size);



def train(model: Rnn, data: DataLoader, epochs: int, optimizer: optim.Optimizer, loss_fn: nn.Module) -> None:

    train_losses = {};
    model.train();
    for epoch in range(epochs):
            epoch_losses = list();
            for X, Y in data:
                # skip batch if it doesnt match with the batch_size
                if X.shape[0] != model.batch_size:
                    continue

                hidden = model.init_hidden(batch_size=model.batch_size)

                # send tensors to device
                X, Y, hidden = X.to(device), Y.to(device), hidden.to(device)

                # 2. clear gradients
                model.zero_grad()

                loss = 0
                print(f'the shape is : {X.shape}');
                for c in range(X.shape[1]):
                    out, hidden = model(X[:, c].reshape(X.shape[0],1), hidden);
                    l = loss_fn(out, Y[:, c].long())
                    loss += l

                # 4. Compte gradients gradients
                loss.backward()

                # 5. Adjust learnable parameters
                # clip as well to avoid vanishing and exploding gradients
                nn.utils.clip_grad_norm_(model.parameters(), 3)
                optimizer.step()
            
                epoch_losses.append(loss.detach().item() / X.shape[1])

            train_losses[epoch] = torch.tensor(epoch_losses).mean()
            # print(f'=> epoch: {epoch + 1}, loss: {train_losses[epoch]}')

            # print(generate_text(model, data.dataset))


        


rnn = Rnn(3 , 128 , 1)
# hidden_tensor = rnn.init_hidden();
# input_tensor = torch.ones(1,3); 
# output, next_hidden = rnn(input_tensor , hidden_tensor);
loss = nn.CrossEntropyLoss()
optimizer = optim.RMSprop(rnn.parameters(), lr = 0.001)
train(rnn,dataLoader,20,optimizer=optimizer,loss_fn=loss);
# print(output,next_hidden)


In [16]:
# x_data = torch.tensor(df.iloc[5]);
df.iloc[5].to_numpy()

array(['2013-01-06', 7.0, 82.8, 1.48, 1018.0], dtype=object)